In [ ]:
import numpy as np
from numpy import sin, cos, exp
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.optimize import fsolve

In [ ]:
save_time = 5

In [ ]:
left_x = -100
right_x = 100
left_y = -100
right_y = 100

# lattice-lattice parameters
m_1 = 1
m_2 = 0.5
c = 0.75
a = 1

# wave packet parameters
gamma = np.pi/6
omega = 1
beta_x = 0.1
beta_y = 0.1
n_0 = -25
v_0 = -35
u_0 = 1

# integration parameters
dt = 0.01
t_max = 85

In [ ]:
num_x = np.round(np.arange(left_x, right_x, a)/a)
num_y = np.round(np.arange(left_y, right_y, a)/a)
mass = np.zeros(shape=(len(num_y),len(num_x)))

for i, num_i in enumerate(num_x):
    for j, num_j in enumerate(num_y):
        if num_i < 0:
            mass[j, i] = m_1
        else:
            mass[j, i] = m_2
mass

In [ ]:
def energy(mass, stiffness, vel, disp):
    e = mass/2 * vel**2 + stiffness/4 * ((np.roll(disp,-1)-disp)**2+(np.roll(disp,1)-disp)**2+\
                                         (np.roll(disp,-1,axis=0)-disp)**2+(np.roll(disp,1,axis=0)-disp)**2)
    return e

In [ ]:
def specify_initial_and_boundary(num_x, num_y, beta_x, beta_y, n_0, v_0, u_0, c, m_1, a, omega):
    disp = np.zeros(shape=(len(num_y),len(num_x)), dtype=float)
    vel = np.zeros(shape=(len(num_y),len(num_x)), dtype=float)

    k_1 = fsolve(lambda k: m_1*omega**2-4*c*(sin(cos(gamma)*k)**2+sin(sin(gamma)*k)**2), 1)[0]*2/a
    g_1 = np.sqrt(c/m_1) * (a*cos(gamma)*cos(cos(gamma)*k_1*a/2)*sin(cos(gamma)*k_1*a/2) - \
                            a*sin(gamma)*cos(sin(gamma)*k_1*a/2)*sin(sin(gamma)*k_1*a/2)) / \
                                (np.sqrt((cos(sin(gamma)*k_1*a/2))**2+(sin(cos(gamma)*k_1*a/2))**2))

    for i, n_x in enumerate(num_x):
        for j, n_y in enumerate(num_y):
            disp[j,i]=u_0*exp(-beta_x**2/2*(n_x*cos(gamma)+n_y*sin(gamma)-n_0*cos(gamma)-v_0*sin(gamma))**2)
            disp[j,i]*=exp(-beta_y**2/2*(-n_x*sin(gamma)+n_y*cos(gamma)+n_0*sin(gamma)-v_0*cos(gamma))**2)
            disp[j,i]*=sin(k_1*a*cos(gamma)*n_x+k_1*a*sin(gamma)*n_y)
            vel[j,i]=-u_0*exp(-beta_x**2/2*(n_x*cos(gamma)+n_y*sin(gamma)-n_0*cos(gamma)-v_0*sin(gamma))**2)
            vel[j,i]*=exp(-beta_y**2/2*(-n_x*sin(gamma)+n_y*cos(gamma)+n_0*sin(gamma)-v_0*cos(gamma))**2)
            vel[j,i]*=(omega*cos(k_1*a*cos(gamma)*n_x+k_1*a*sin(gamma)*n_y)-\
                    beta_x**2*g_1/a*(n_x*cos(gamma)+n_y*sin(gamma)-n_0*cos(gamma))*\
                        sin(k_1*a*cos(gamma)*n_x+k_1*a*sin(gamma)*n_y))

    return disp, vel

In [ ]:
def solver(mass, disp, vel, c, disp_history, vel_history, energy_history, t_max, dt, save_time):

    times = np.arange(0, t_max, dt)

    for t in tqdm(times):

        if t % save_time == 0:
            disp_history += [disp]
            vel_history += [vel]
            energy_history += [energy(mass, c, vel, disp)]
        
        # leapfrog synchronized form
        acc1 = (c/mass)*(np.roll(disp,-1)+np.roll(disp,1)+np.roll(disp,-1,axis=0)+np.roll(disp,1,axis=0)-4*disp)
        disp += vel*dt+1/2*acc1*dt**2
        acc2 = (c/mass)*(np.roll(disp,-1)+np.roll(disp,1)+np.roll(disp,-1,axis=0)+np.roll(disp,1,axis=0)-4*disp)
        vel += 1/2*(acc1+acc2)*dt

        # leapfrog self-made form
        #deform = np.roll(disp,-1) + np.roll(disp,1) + np.roll(disp,-1,axis=0) + np.roll(disp,1,axis=0) - 4*disp
        #vel += (c/mass)*deform*dt
        #disp += vel * dt

    return disp_history, vel_history, energy_history

In [ ]:
disp, vel = specify_initial_and_boundary(num_x, num_y, beta_x, beta_y, n_0, v_0, u_0, c, m_1, a, omega)

fig = plt.figure()
ax = plt.axes(projection="3d")
x, y = np.meshgrid(num_x, num_y)
surf = ax.plot_surface(x, y, disp)
plt.show()

In [ ]:
disp_history = []
vel_history = []
energy_history = []
disp_history, vel_history, energy_history = solver(mass, disp, vel, c, disp_history, vel_history,
                                                   energy_history, t_max, dt, save_time)

fig = plt.figure()
ax = plt.axes(projection="3d")
x, y = np.meshgrid(num_x, num_y)
surf = ax.plot_surface(x, y, disp)
plt.show()

In [ ]:
X, Y = np.meshgrid(num_x, num_y)
levels = np.linspace(disp.min(), disp.max(), 100)
fig, ax = plt.subplots()
cs = ax.contourf(X, Y, disp,levels=levels)
cbar = fig.colorbar(cs)
plt.show()

In [ ]:
#%matplotlib qt
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
X, Y = np.meshgrid(num_x, num_y)
surf = ax.plot_surface(X, Y, disp, cmap=cm.cool)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

fig, ax = plt.subplots()
X, Y = np.meshgrid(num_x, num_y)
levels = np.linspace(energy_history[0].min(), energy_history[0].max(), 100)

cf = ax.contourf(X, Y, energy_history[0], levels=levels)
cbar = fig.colorbar(cf)
plt.grid()
plt.plot([0,0],[num_y[0],num_y[-1]], c='r', linewidth=0.8)
plt.xlabel('Номер частицы по оси Ox')
plt.ylabel('Номер частицы по оси Oy')

def update(frame):
    cf = ax.contourf(X, Y, energy_history[frame], levels=levels)
    plt.title(f't={5*frame}')
    return cf, 

animation.FuncAnimation(fig, update, frames=len(energy_history))

TODO: изменение colorbar в анимации: [https://stackoverflow.com/questions/39472017/how-to-animate-the-colorbar-in-matplotlib](https://stackoverflow.com/questions/39472017/how-to-animate-the-colorbar-in-matplotlib).